make figure showing:

- Merapi true colour
- Merapi slope
- Merapi shadow overlay map
- interferogram
- pixel offset range and azimuth


In [1]:
# import external packages
import numpy as np
import pandas as pd
import numba
from numba import vectorize
import glob # for file search
import copy
import os # operating system stuff
import re # regex
import fastparquet # fast read/write for large data structures
import sklearn.preprocessing as pre # for data normalisation
from sklearn.metrics import pairwise_distances

import geopandas as gpd
import rasterio as rio
import rasterio.mask
from rasterio.plot import plotting_extent
from shapely.geometry import Polygon
from shapely.geometry.point import Point
import pyproj
from pyproj import CRS
from inpoly import inpoly2 # for fast inpolygon checks
import utm

import matplotlib.pyplot as plt 
import matplotlib.dates as mdates
from matplotlib import cm as mpl_cm
from matplotlib import colors as mcolors 
import matplotlib.image as mplimg

from mpl_toolkits.axes_grid1 import make_axes_locatable # for colorbar scaling
from mpl_toolkits.axes_grid1 import ImageGrid
from matplotlib_scalebar.scalebar import ScaleBar
from matplotlib.gridspec import GridSpec
from matplotlib.ticker import FormatStrFormatter

import seaborn as sns
from matplotlib import rc_file_defaults
rc_file_defaults()
# sns.set(style=None, color_codes=True)

from shapely.geometry import Polygon
from shapely.geometry.point import Point
import datetime

import configparser

from cmcrameri import cm # for scientific colourmaps

###########################
# import main local package
import SPOTSAR_main as sm


In [2]:
################ Define user INPUTS #######################
######## please edit the values of this block only ########
###########################################################

# define hillshade file
HS_FILE = './test_data/DEM/TDX_Merapi_WGS84_HS.tif'

# Indonesia shapefile
IND_COAST = './test_data/DEM/IDN_adm/IDN_adm0.shp' 

# define lon and lat files
LON_FILE = './test_data/CSK_dsc/geo2/20200910.lon'
LAT_FILE = './test_data/CSK_dsc/geo2/20200910.lat'

S1_LON_FILE = './test_data/S1_dsc/geo/20200616.lon'
S1_LAT_FILE = './test_data/S1_dsc/geo/20200616.lat'

# define parameter text file
PARAM_FILE = './test_data/CSK_dsc/params.txt'

# DEM, SLOPE and NDVI file
DEM_FILE = './test_data/DEM/TDX_Merapi_WGS84_5m.tif'
SLOPE_FILE = './test_data/DEM/TDX_Merapi_WGS84_5m_slope.tif'
NDVI_FILE = '/Users/markbemelmans/Documents/PhD/projects/Merapi2021/sentinel2/Sentinel_NDVI_WGS84_v2.tif'
TRUE_COL_FILE = '/Users/markbemelmans/Documents/PhD/projects/Merapi2021/sentinel2/Sentinel_true_color_WGS84.tif'
LS_MAP_FILE = '/Users/markbemelmans/Documents/PhD/projects/Merapi2021/CSK/dsc1/c20200910.ls_map_dem_seg.png'
IFG_FILE1 = '/Users/markbemelmans/Documents/PhD/projects/Merapi2021/CSK/dsc1/example_ifgs/20200927_20201113_img.diff.gc.tif'
IFG_FILE2 = '/Users/markbemelmans/Documents/PhD/projects/Merapi2021/CSK/dsc1/example_ifgs/20200926_20201113_img.diff.gc.tif'
IFG_FILE3 = '/Users/markbemelmans/Documents/PhD/projects/Merapi2021/CSK/dsc1/example_ifgs/20200927_20210812_img.diff.gc.tif'
IFG_FILE4 = '/Users/markbemelmans/Documents/PhD/projects/Merapi2021/CSK/dsc1/example_ifgs/20210217_20210218_img.diff.gc.tif'
# PWR_FILE = './test_data/CSK_dsc/rmlis/c20200910.rmli.gc.tif'
PWR_FILE = './test_data/CSK_dsc/rmlis/c20200910.crop.rmli.ras'

S1_IFG_FILE = './test_data/S1_dsc/20200908_20201119/20200908_20201119.diff.gc.tif'
S1_SPOT_FILE = './test_data/S1_dsc/20200908_20201119/disp_20200908_20201119.txt'
S1_SPOT_FILE_CCS = './test_data/S1_dsc/20200908_20201119/20200908_20201119.ccs'
# S1_PWR_FILE = './test_data/S1_dsc/20200908_20201119/20200908.rmli.gc.tif'
S1_PWR_FILE = './test_data/S1_dsc/20200908_20201119/20200908.crop.rmli.ras'

# lava flow files:
L1888_FILE = '/Users/markbemelmans/Documents/PhD/projects/Merapi2021/merapi_maps/L1888_v2.shp'
L1992_FILE = '/Users/markbemelmans/Documents/PhD/projects/Merapi2021/merapi_maps/L1992.shp'
L1998_FILE = '/Users/markbemelmans/Documents/PhD/projects/Merapi2021/merapi_maps/L1998.shp'

# define map region of interest
lon_lims = [110.41, 110.45]
lat_lims = [-7.555, -7.535]



# define colour range {min max} (min = -max)
vmax = 3 # range of colourscale in meters

# define file names for data, lon and lat
DIRECTORY_PATH = "./test_data/CSK_dsc/DISP_txt2/"
# define path to ccp and ccs files
DIRECTORY_PATH_CCS = "./test_data/CSK_dsc/CCS2/"

# Set the regular expression pattern to match the file names
PATTERN1 = r"^c20200927_c20201113_disp_[0-9]+_[0-9]+\.txt$"
PATTERN2 = r"^c20200926_c20201113_disp_[0-9]+_[0-9]+\.txt$"
PATTERN3 = r"^c20200927_c20210812_disp_[0-9]+_[0-9]+\.txt$"
PATTERN4 = r"^c20210217_c20210218_disp_[0-9]+_[0-9]+\.txt$"

# Set the regular expression pattern to match the ccs file names
PATTERN_CCS1 = r"^c20200927_c20201113_ccs_[0-9]+_[0-9]+$"
PATTERN_CCS2 = r"^c20200926_c20201113_ccs_[0-9]+_[0-9]+$"
PATTERN_CCS3 = r"^c20200927_c20210812_ccs_[0-9]+_[0-9]+$"
PATTERN_CCS4 = r"^c20210217_c20210218_ccs_[0-9]+_[0-9]+$"


# open hillshade file and re-order offset and CCS files

# open hill shade file with rasterio
DEM_HS = rio.open(HS_FILE)
SHADING = DEM_HS.read(1,masked=True) # rasterio bands are indexed from 1

# extract DEM extent
DEM_EXTENT=[DEM_HS.bounds.left,DEM_HS.bounds.right,DEM_HS.bounds.bottom,DEM_HS.bounds.top]

# read ls map
ls_map = mplimg.imread(LS_MAP_FILE)

# get world borders via geopandas

world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

# read parameters from text file
config = configparser.ConfigParser()
config.read(PARAM_FILE)
WIDTH = int(config.get('params', 'width'))
LINES = int(config.get('params', 'lines'))
WIDTH_CCS = int(config.get('params', 'width_ccs'))
LINES_CCS = int(config.get('params', 'lines_ccs'))
R_START = int(config.get('params', 'r_start'))
A_START = int(config.get('params', 'a_start'))
R_STEP = int(config.get('params', 'r_step'))
A_STEP = int(config.get('params', 'a_step'))
HEADING = float(config.get('params', 'heading'))
MEAN_INC = float(config.get('params', 'mean_inc'))

S1_WIDTH = 24910
S1_LINES = 4185
S1_PWR_WIDTH = 2310
S1_PWR_LINES = 1215
S1_WIDTH_CCS = 262
S1_LINES_CCS = 546
S1_R_START = 2804
S1_A_START = 1743
S1_R_STEP = 10
S1_A_STEP = 2
S1_HEADING = 192.0799295
S1_MEAN_INC = 39.1333

In [3]:
# reorder file using Post_processing.reorder_files
matching_files1 = sm.Post_processing.reorder_files(DIRECTORY_PATH,PATTERN1,0)
matching_files_ccs1 = sm.Post_processing.reorder_files(DIRECTORY_PATH_CCS,PATTERN_CCS1,0)
matching_files2 = sm.Post_processing.reorder_files(DIRECTORY_PATH,PATTERN2,0)
matching_files_ccs2 = sm.Post_processing.reorder_files(DIRECTORY_PATH_CCS,PATTERN_CCS2,0)
matching_files3 = sm.Post_processing.reorder_files(DIRECTORY_PATH,PATTERN3,0)
matching_files_ccs3 = sm.Post_processing.reorder_files(DIRECTORY_PATH_CCS,PATTERN_CCS3,0)
matching_files4 = sm.Post_processing.reorder_files(DIRECTORY_PATH,PATTERN4,0)
matching_files_ccs4 = sm.Post_processing.reorder_files(DIRECTORY_PATH_CCS,PATTERN_CCS4,0)


# test if file ordering has worked
print(matching_files1)
print(matching_files_ccs1)
print(matching_files2)
print(matching_files_ccs2)
print(matching_files3)
print(matching_files_ccs3)
print(matching_files4)
print(matching_files_ccs4)


['c20200927_c20201113_disp_58_28.txt', 'c20200927_c20201113_disp_140_68.txt', 'c20200927_c20201113_disp_224_108.txt', 'c20200927_c20201113_disp_306_148.txt', 'c20200927_c20201113_disp_388_188.txt']
['c20200927_c20201113_ccs_58_28', 'c20200927_c20201113_ccs_140_68', 'c20200927_c20201113_ccs_224_108', 'c20200927_c20201113_ccs_306_148', 'c20200927_c20201113_ccs_388_188']
['c20200926_c20201113_disp_58_28.txt', 'c20200926_c20201113_disp_140_68.txt', 'c20200926_c20201113_disp_224_108.txt', 'c20200926_c20201113_disp_306_148.txt', 'c20200926_c20201113_disp_388_188.txt']
['c20200926_c20201113_ccs_58_28', 'c20200926_c20201113_ccs_140_68', 'c20200926_c20201113_ccs_224_108', 'c20200926_c20201113_ccs_306_148', 'c20200926_c20201113_ccs_388_188']
['c20200927_c20210812_disp_58_28.txt', 'c20200927_c20210812_disp_140_68.txt', 'c20200927_c20210812_disp_224_108.txt', 'c20200927_c20210812_disp_306_148.txt', 'c20200927_c20210812_disp_388_188.txt']
['c20200927_c20210812_ccs_58_28', 'c20200927_c20210812_ccs_1

In [4]:
# load data from files into class multi-kernel
example_pairs = []
for (matching_files,matching_files_ccs) in zip([matching_files1,matching_files2,matching_files3,matching_files4],[matching_files_ccs1,matching_files_ccs2,matching_files_ccs3,matching_files_ccs4]):
    datastack = sm.Post_processing.MultiKernel(DIRECTORY_PATH,
                                            matching_files,
                                            DIRECTORY_PATH_CCS,
                                            matching_files_ccs,
                                            LAT_FILE,
                                            LON_FILE,
                                            HEADING,
                                            MEAN_INC,
                                            LINES_CCS,
                                            WIDTH_CCS)
    # We need to assign some data not stored in the disp.txt files.
    datastack.get_params_from_file_name()
    datastack.get_latlon_from_file(WIDTH)
    datastack.add_lat_lon_to_data(R_START,A_START)
    datastack.crop_stack_ccs(R_STEP,A_STEP)
    # the object datastack now has several attributes associated with the whole dataset (e.g., date1, date2, heading)
    # Next we add all the offset data (disp.txt) to the stack
    stacked_data = datastack.assign_data_to_stack(R_STEP,A_STEP)
    # The attribute 'Stack' we find a list of single-kernel objects which contain the actual offset data, ccp and ccs data and the coordinates.

    # add stack to list
    example_pairs.append(datastack)



In [5]:

## Sentinel-1 offset tracking pair

S1_data = pd.read_csv(S1_SPOT_FILE, header  =None, sep = '\s+')

# get latitude and longitude data from file
S1_lon_vec = np.fromfile(S1_LON_FILE, dtype='>f', count=-1)
S1_lat_vec = np.fromfile(S1_LAT_FILE, dtype='>f', count=-1)
# set 0,0 coordinates to nan
S1_lon_vec[S1_lon_vec == 0] = np.nan
S1_lat_vec[S1_lat_vec == 0] = np.nan
# get number of lines in lat lon files
lines_ll = S1_lon_vec.shape[0]/S1_WIDTH
# reshape tomatrix for index referencing
S1_LON = np.reshape(S1_lon_vec,[int(lines_ll),S1_WIDTH])
S1_LAT = np.reshape(S1_lat_vec,[int(lines_ll),S1_WIDTH]) 

S1_data[9]  = S1_LAT[S1_data[1],S1_data[0]]
S1_data[10] = S1_LON[S1_data[1],S1_data[0]]

print(S1_data)


         0     1      2      3      4      5      6      7       8         9   \
0      3294  1909 -0.100  0.064 -0.121 -0.026  0.404  0.049   1.266 -7.497390   
1      3304  1909 -0.095  0.076 -0.121 -0.026  0.398  0.060   1.425 -7.497278   
2      3314  1909 -0.109  0.079 -0.120 -0.026  0.389  0.027   1.477 -7.497250   
3      3284  1911 -0.117  0.047 -0.121 -0.026  0.412  0.009   1.016 -7.497804   
4      3294  1911 -0.104  0.061 -0.121 -0.026  0.407  0.040   1.214 -7.497621   
...     ...   ...    ...    ...    ...    ...    ...    ...     ...       ...   
66633  4824  2669 -0.900 -0.582 -0.187  0.016  0.138 -1.660  -8.406 -7.580797   
66634  4834  2669  0.428 -0.716 -0.189  0.016  0.127  1.435 -10.277 -7.580739   
66635  4794  2671 -0.303 -0.210 -0.183  0.016  0.181 -0.278  -3.179 -7.581275   
66636  4804  2671 -0.435 -0.375 -0.185  0.016  0.159 -0.582  -5.499 -7.581219   
66637  4814  2671 -0.788 -0.502 -0.186  0.016  0.141 -1.402  -7.280 -7.581149   

               10  
0      

In [6]:
import fiona
from matplotlib.colors import ListedColormap
from matplotlib import cm as mpl_cm
## find nearest elevation, slope, and NDVI values


def read_tiff(filename,n_bands):
    with rio.open(filename) as src:

        values = [src.read(i+1) for i in range(n_bands)]
        if n_bands > 1:
            values_stack = np.stack(values,axis=-1)
        else:
            values_stack = values[0]
        print(f'Data {filename} has shape', values_stack.shape)
        height = values_stack.shape[0]
        width = values_stack.shape[1]
        cols, rows = np.meshgrid(np.arange(width), np.arange(height))
        xs, ys = rio.transform.xy(src.transform, rows, cols)
        lons = np.array(xs)
        lats = np.array(ys)
        extent = [src.bounds.left,src.bounds.right,src.bounds.bottom,src.bounds.top]
    return values_stack, lons, lats, extent 


# load indonesia borders
ind_coastlines = gpd.read_file(IND_COAST)


DEM_heights, DEM_lons, DEM_lats, DEM_extent = read_tiff(DEM_FILE,1)
DEM_slope, slope_lons, slope_lats, slope_extent = read_tiff(SLOPE_FILE,1)

## NDVI open and crop data to DEM extent
# define map region of interest
lon_lims = [np.nanmin(DEM_lons), np.nanmax(DEM_lons)]
lat_lims = [np.nanmin(DEM_lats), np.nanmax(DEM_lats)]

# define map region of interest
lon_lims = [110.41, 110.46]
lat_lims = [-7.555, -7.515]

crop_flag=1


# create cropping polygon from ROI
if crop_flag:
    coords = ((lon_lims[0], lat_lims[0]), (lon_lims[0], lat_lims[1]), (lon_lims[1], lat_lims[1]), (lon_lims[1], lat_lims[0]), (lon_lims[0], lat_lims[0]))
    crop_poly = Polygon(coords)
    crop_poly_geojson = gpd.GeoSeries([crop_poly])
    crop_poly_geojson.to_file('./test_data/crop_ndvi_poly.shp',crs="EPSG:4326")


with fiona.open('./test_data/crop_ndvi_poly.shp', "r") as shapefile:
    shapes = [feature["geometry"] for feature in shapefile]

# get data and read coords from first file
with rio.open(NDVI_FILE) as src:
    out_image, out_transform = rasterio.mask.mask(src, shapes, crop=True)
    out_meta = src.meta
    out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})
with rasterio.open("./test_Data/ndvi_cropped.tif", "w", **out_meta) as dest:
    dest.write(out_image)

NDVI, ndvi_lons, ndvi_lats, ndvi_extent = read_tiff("./test_Data/ndvi_cropped.tif",1)


with rio.open(TRUE_COL_FILE) as src:
    out_image, out_transform = rasterio.mask.mask(src, shapes, crop=True)
    out_meta = src.meta
    out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})
with rasterio.open("./test_Data/TRUE_COL_cropped.tif", "w", **out_meta) as dest:
    dest.write(out_image)

TRUE_COL, TRUE_COL_lons, TRUE_COL_lats, TRUE_COL_extent = read_tiff("./test_Data/TRUE_COL_cropped.tif",3)
CSK_IFG1, CSK_IFG_lons1, CSK_IFG_lats1, CSK_IFG_extent1 = read_tiff(IFG_FILE1,3)
CSK_IFG2, CSK_IFG_lons2, CSK_IFG_lats2, CSK_IFG_extent2 = read_tiff(IFG_FILE2,3)
CSK_IFG3, CSK_IFG_lons3, CSK_IFG_lats3, CSK_IFG_extent3 = read_tiff(IFG_FILE3,3)
CSK_IFG4, CSK_IFG_lons4, CSK_IFG_lats4, CSK_IFG_extent4 = read_tiff(IFG_FILE4,3)
# CSK_PWR, CSK_PWR_lons, CSK_PWR_lats, CSK_PWR_extent = read_tiff(PWR_FILE,1)
# with rio.open(PWR_FILE) as src:
#     CSK_PWR = src.read(1)

CSK_PWR = plt.imread(PWR_FILE)


S1_IFG, S1_IFG_lons, S1_IFG_lats, S1_IFG_extent = read_tiff(S1_IFG_FILE,3)
# S1_PWR, S1_PWR_lons, S1_PWR_lats, S1_PWR_extent = read_tiff(S1_PWR_FILE,1)
# with rio.open(S1_PWR_FILE) as src:
#     S1_PWR = src.read(1)
S1_PWR = plt.imread(S1_PWR_FILE)

# with rio.open("./test_Data/TRUE_COL_cropped.tif") as src:
#     TRUE_COL_1 = src.read(1)
#     TRUE_COL_2 = src.read(2)
#     TRUE_COL_3 = src.read(3)
#     print('Band1 has shape', TRUE_COL_1.shape)
#     height = NDVI.shape[0]
#     width = NDVI.shape[1]
#     cols, rows = np.meshgrid(np.arange(width), np.arange(height))
#     xs, ys = rio.transform.xy(src.transform, rows, cols)
#     TRUE_COL_lons = np.array(xs)
#     TRUE_COL_lats = np.array(ys)
#     TRUE_COL_extent = [src.bounds.left,src.bounds.right,src.bounds.bottom,src.bounds.top]

# TRUE_COL = np.stack((TRUE_COL_1,TRUE_COL_2,TRUE_COL_3),axis=-1)


# with rio.open(IFG_FILE) as src:
#     IFG_1 = src.read(1)
#     IFG_2 = src.read(2)
#     IFG_3 = src.read(3)
#     print('Band1 has shape', IFG_1.shape)
#     height = DEM_heights.shape[0]
#     width = DEM_heights.shape[1]
#     cols, rows = np.meshgrid(np.arange(width), np.arange(height))
#     xs, ys = rio.transform.xy(src.transform, rows, cols)
#     IFG_lons = np.array(xs)
#     IFG_lats = np.array(ys)
#     IFG_extent = [src.bounds.left,src.bounds.right,src.bounds.bottom,src.bounds.top]

# IFG = np.stack((IFG_1,IFG_2,IFG_3),axis=-1)

x= mpl_cm.get_cmap('Blues_r', 135)
y= mpl_cm.get_cmap('YlGn', 135)
z = np.vstack((x(range(135)),
                       y(range(135))))
ndvi_cmap = ListedColormap(z, name='BlYlGn')

Data ./test_data/DEM/TDX_Merapi_WGS84_5m.tif has shape (2028, 1672)
Data ./test_data/DEM/TDX_Merapi_WGS84_5m_slope.tif has shape (2028, 1672)
Data ./test_Data/ndvi_cropped.tif has shape (75, 93)
Data ./test_Data/TRUE_COL_cropped.tif has shape (442, 553, 3)
Data /Users/markbemelmans/Documents/PhD/projects/Merapi2021/CSK/dsc1/example_ifgs/20200927_20201113_img.diff.gc.tif has shape (10171, 8384, 3)
Data /Users/markbemelmans/Documents/PhD/projects/Merapi2021/CSK/dsc1/example_ifgs/20200926_20201113_img.diff.gc.tif has shape (10171, 8384, 3)
Data /Users/markbemelmans/Documents/PhD/projects/Merapi2021/CSK/dsc1/example_ifgs/20200927_20210812_img.diff.gc.tif has shape (10171, 8384, 3)
Data /Users/markbemelmans/Documents/PhD/projects/Merapi2021/CSK/dsc1/example_ifgs/20210217_20210218_img.diff.gc.tif has shape (10171, 8384, 3)
Data ./test_data/S1_dsc/20200908_20201119/20200908_20201119.diff.gc.tif has shape (10171, 8384, 3)


/var/folders/m9/32v45xkn4zx6js6htb1prb2r0000gn/T/ipykernel_32838/413428283.py:129: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  x= mpl_cm.get_cmap('Blues_r', 135)
/var/folders/m9/32v45xkn4zx6js6htb1prb2r0000gn/T/ipykernel_32838/413428283.py:130: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  y= mpl_cm.get_cmap('YlGn', 135)


In [7]:
import matplotlib.patches as mpatches
import matplotlib.transforms as mtransforms

def add_right_cax(ax, pad, width):
    axpos = ax.get_position()
    caxpos = mtransforms.Bbox.from_extents(
        axpos.x1 + pad,
        axpos.y0,
        axpos.x1 + pad + width,
        axpos.y1
    )
    cax = ax.figure.add_axes(caxpos)

    return cax

from matplotlib.colors import LinearSegmentedColormap
clist = ['#f579cd', '#f67fc6', '#f686bf', '#f68cb9', '#f692b3', '#f698ad',
                     '#f69ea7', '#f6a5a1', '#f6ab9a', '#f6b194', '#f6b78e', '#f6bd88',
                     '#f6c482', '#f6ca7b', '#f6d075', '#f6d66f', '#f6dc69', '#f6e363',
                     '#efe765', '#e5eb6b', '#dbf071', '#d0f477', '#c8f67d', '#c2f684',
                     '#bbf68a', '#b5f690', '#aff696', '#a9f69c', '#a3f6a3', '#9cf6a9',
                     '#96f6af', '#90f6b5', '#8af6bb', '#84f6c2', '#7df6c8', '#77f6ce',
                     '#71f6d4', '#6bf6da', '#65f6e0', '#5ef6e7', '#58f0ed', '#52e8f3',
                     '#4cdbf9', '#7bccf6', '#82c4f6', '#88bdf6', '#8eb7f6', '#94b1f6',
                     '#9aabf6', '#a1a5f6', '#a79ef6', '#ad98f6', '#b392f6', '#b98cf6',
                     '#bf86f6', '#c67ff6', '#cc79f6', '#d273f6', '#d86df6', '#de67f6',
                     '#e561f6', '#e967ec', '#ed6de2', '#f173d7']
colormap = LinearSegmentedColormap.from_list('dismph', clist, N=256)
#colormap = self.cmap_map(lambda x: x/2 + 0.5, colormap)  # brighten colormap
#colormap = self.cmap_map(lambda x: x*0.75, colormap)     # darken colormap
colormap.set_bad('w', 0.0)

dismph = colormap(range(255))
dismph2 = np.row_stack((np.array(dismph),np.array(dismph)))
print(dismph2)

colormap = LinearSegmentedColormap.from_list('dismph2', dismph2, N=256)
#colormap = self.cmap_map(lambda x: x/2 + 0.5, colormap)  # brighten colormap
#colormap = self.cmap_map(lambda x: x*0.75, colormap)     # darken colormap
colormap.set_bad('w', 0.0)

[[0.96078431 0.4745098  0.80392157 1.        ]
 [0.96175317 0.48032295 0.79713956 1.        ]
 [0.96272203 0.4861361  0.79035755 1.        ]
 ...
 [0.93347174 0.43354095 0.87510957 1.        ]
 [0.93734717 0.43935409 0.86445213 1.        ]
 [0.94122261 0.44516724 0.85379469 1.        ]]


In [12]:
%matplotlib osx
## overview figure

# 1   4   7  10  13 16 
# 2   5   8  11  14 17
# 3   6   9  12  15 18

# 1: True colour S2
# 2: CSK Amplitude
# 3: S1 Amplitude
# 4: S1 ifg
# 5: S1 range off
# 6: S1 azi off
# 7,10,13,16: CSK ifg
# 8,11,14,17: CSK range off
# 9,12,15,18: CSK azi off

from matplotlib.patches import Rectangle
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
# make figure
grid_size = 1000
cmap=cm.vik
clims = [-3, 3]
# define map region of interest
lon_lims = [110.425, 110.46]
lat_lims = [-7.555, -7.515]
orientation = 'vertical'

CSK_obj1 = example_pairs[0].Stack[1]
CSK_obj2 = example_pairs[1].Stack[1]
CSK_obj3 = example_pairs[2].Stack[1]
CSK_obj4 = example_pairs[3].Stack[1]


merapi_fig, merapi_ax = plt.subplots(1,1)

# true colour
merapi_ax.imshow(TRUE_COL,extent=TRUE_COL_extent)
merapi_ax.scatter(x=[110.410626],y=[-7.526175],s=20,marker='s',color='tab:blue', label= 'Pos Badaban')
merapi_ax.scatter(x=[110.439757],y=[-7.537567],s=20,marker='v',color='tab:orange', label='EDM reflector')
merapi_ax.scatter(x=[110.440165],y=[-7.537451],s=20,marker='v',color='tab:orange')
merapi_ax.scatter(x=[110.439260],y=[-7.537533],s=20,marker='v',color='tab:orange')
merapi_ax.plot(x=[lon_lims[0],lon_lims[0],lon_lims[1],lon_lims[1],lon_lims[0]],y=[lat_lims[0],lat_lims[1],lat_lims[1],lat_lims[0],lat_lims[0]],linewidth=2,color='red',label='Figure x outlier')
merapi_ax.legend(location='best')

merapi_ax_1 = merapi_ax.inset_axes([0.4,0.70,0.6,0.3])
merapi_ax_1.add_patch(Rectangle((90,-15),55,25,color='white'))
ind_coastlines.plot(ax=merapi_ax_1,color='lightgrey',edgecolor='black')
merapi_ax_1.scatter(110.4,-7.54,s=5,c='red')
merapi_ax_1.set_aspect('equal','box')
merapi_ax_1.set_xlim((105,117))
merapi_ax_1.set_ylim((-9,-5))
merapi_ax_1.set_axis_off()

merapi_ax.set_aspect('equal','box')
merapi_ax.set_xlim([110.41, 110.46])
merapi_ax.set_ylim(-7.555, -7.515)
merapi_ax.add_artist(ScaleBar(sm.plot.get_1deg_dist(),location='upper left',font_properties={
    "family": "serif",
    "size": "large",
}))


fig = plt.figure()

gs = GridSpec(3, 6, figure=fig)
ax00 = fig.add_subplot(gs[0, 0]) # Merapi true colour with inset
ax10 = fig.add_subplot(gs[1, 0]) # CSK amplitude (with summit inset?)
ax20 = fig.add_subplot(gs[2, 0]) # S1 amplitude (with summit inset?)
ax01 = fig.add_subplot(gs[0, 1]) # S1 ifg
ax11 = fig.add_subplot(gs[1, 1]) # S1 range off
ax21 = fig.add_subplot(gs[2, 1]) # S1 azi off
ax02 = fig.add_subplot(gs[0, 2]) # CSK1 ifg
ax12 = fig.add_subplot(gs[1, 2]) # CSK1 range off
ax22 = fig.add_subplot(gs[2, 2]) # CSK1 azi off
ax03 = fig.add_subplot(gs[0, 3]) # CSK2 ifg
ax13 = fig.add_subplot(gs[1, 3]) # CSK2 range off
ax23 = fig.add_subplot(gs[2, 3]) # CSK2 azi off
ax04 = fig.add_subplot(gs[0, 4]) # CSK3 ifg
ax14 = fig.add_subplot(gs[1, 4]) # CSK3 range off
ax24 = fig.add_subplot(gs[2, 4]) # CSK3 azi off
ax05 = fig.add_subplot(gs[0, 5]) # CSK4 ifg
ax15 = fig.add_subplot(gs[1, 5]) # CSK4 range off
ax25 = fig.add_subplot(gs[2, 5]) # CSK4 azi off


# true colour
ax00.imshow(TRUE_COL,extent=TRUE_COL_extent)
ax00.scatter(x=[110.410626],y=[-7.526175],s=20,marker='s',color='tab:blue')
# ax00.scatter(x=[110.439757],y=[-7.537567],s=20,marker='v',color='tab:orange')
ax00.scatter(x=[110.440165],y=[-7.537451],s=20,marker='v',color='tab:orange')
# ax00.scatter(x=[110.439260],y=[-7.537533],s=20,marker='v',color='tab:orange')

ax00_1 = ax00.inset_axes([0.4,0.77,0.6,0.3])
ax00_1.add_patch(Rectangle((90,-15),55,25,color='white'))
# world.plot(ax=ax00_1,color='lightgrey',edgecolor='black')
ind_coastlines.plot(ax=ax00_1,color='lightgrey',edgecolor='black')
ax00_1.scatter(110.4,-7.54,s=5,c='red')
ax00_1.set_aspect('equal','box')
ax00_1.set_xlim((105,117))
ax00_1.set_ylim((-9,-5))
ax00_1.set_axis_off()

# S1 pwr
ax10.imshow(S1_PWR[470:675,1075:1335],cmap='Greys')

# CSK pwr
ax20.imshow(CSK_PWR[3400:5710,1440:3770],cmap='Greys')

# S1 ifg
ax01.imshow(S1_IFG,extent=S1_IFG_extent)

# S1 r off
S1_r_off_map = ax11.hexbin(S1_data[10],S1_data[9],C=S1_data[7],gridsize=grid_size,cmap=cmap,vmin=clims[0],vmax=clims[1])

# S1 a off
S1_a_off_map = ax21.hexbin(S1_data[10],S1_data[9],C=S1_data[8],gridsize=grid_size,cmap=cmap,vmin=clims[0],vmax=clims[1])


# CSK ifg 1
ax02.imshow(CSK_IFG1,extent=CSK_IFG_extent1)

# CSK r off 1
CSK_r_off_map = ax12.hexbin(CSK_obj1.Lon_off.flatten(),CSK_obj1.Lat_off.flatten(),C=CSK_obj1.R_off.flatten(),gridsize=grid_size,cmap=cmap,vmin=clims[0],vmax=clims[1])
# plt.colorbar(r_off_map,label='range offset [m]',orientation=orientation)

# CSK a off 1
CSK_a_off_map = ax22.hexbin(CSK_obj1.Lon_off.flatten(),CSK_obj1.Lat_off.flatten(),C=CSK_obj1.A_off.flatten(),gridsize=grid_size,cmap=cmap,vmin=clims[0],vmax=clims[1])
# plt.colorbar(a_off_map,label='azimuth offset [m]',orientation=orientation)


# CSK ifg 2
ax03.imshow(CSK_IFG2,extent=CSK_IFG_extent2)

# CSK r off 2
CSK_r_off_map = ax13.hexbin(CSK_obj2.Lon_off.flatten(),CSK_obj2.Lat_off.flatten(),C=CSK_obj2.R_off.flatten(),gridsize=grid_size,cmap=cmap,vmin=clims[0],vmax=clims[1])
# plt.colorbar(r_off_map,label='range offset [m]',orientation=orientation)

# CSK a off 2
CSK_a_off_map = ax23.hexbin(CSK_obj2.Lon_off.flatten(),CSK_obj2.Lat_off.flatten(),C=CSK_obj2.A_off.flatten(),gridsize=grid_size,cmap=cmap,vmin=clims[0],vmax=clims[1])
# plt.colorbar(a_off_map,label='azimuth offset [m]',orientation=orientation)


# CSK ifg 3
ax04.imshow(CSK_IFG3,extent=CSK_IFG_extent3)

# CSK r off 3
CSK_r_off_map = ax14.hexbin(CSK_obj3.Lon_off.flatten(),CSK_obj3.Lat_off.flatten(),C=CSK_obj3.R_off.flatten(),gridsize=grid_size,cmap=cmap,vmin=clims[0],vmax=clims[1])
# plt.colorbar(r_off_map,label='range offset [m]',orientation=orientation)

# CSK a off 3
CSK_a_off_map = ax24.hexbin(CSK_obj3.Lon_off.flatten(),CSK_obj3.Lat_off.flatten(),C=CSK_obj3.A_off.flatten(),gridsize=grid_size,cmap=cmap,vmin=clims[0],vmax=clims[1])
# plt.colorbar(a_off_map,label='azimuth offset [m]',orientation=orientation)


# CSK ifg 4
ax05.imshow(CSK_IFG4,extent=CSK_IFG_extent4)

# CSK r off 4
CSK_r_off_map = ax15.hexbin(CSK_obj4.Lon_off.flatten(),CSK_obj4.Lat_off.flatten(),C=CSK_obj4.R_off.flatten(),gridsize=grid_size,cmap=cmap,vmin=clims[0],vmax=clims[1])
# plt.colorbar(r_off_map,label='range offset [m]',orientation=orientation)

# CSK a off 4
CSK_a_off_map = ax25.hexbin(CSK_obj4.Lon_off.flatten(),CSK_obj4.Lat_off.flatten(),C=CSK_obj4.A_off.flatten(),gridsize=grid_size,cmap=cmap,vmin=clims[0],vmax=clims[1])
# plt.colorbar(a_off_map,label='azimuth offset [m]',orientation=orientation)

for a in [ax00,ax01,ax11,ax21,ax02,ax12,ax22,ax03,ax13,ax23,ax04,ax14,ax24,ax05,ax15,ax25]:
    a.set_aspect('equal','box')
    a.set_xlim(lon_lims)
    a.set_ylim(lat_lims)
    a.add_artist(ScaleBar(sm.plot.get_1deg_dist(),location='upper left',font_properties={
        "family": "serif",
        "size": "large",
    }))

for (a,s) in zip([ax00,ax01,ax11,ax21,ax02,ax12,ax22,ax03,ax13,ax23,ax04,ax14,ax24,ax05,ax15,ax25],['A','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R']):
    a.annotate(s,xy=[lon_lims[0]+0.002,lat_lims[0]+0.002],xytext=[lon_lims[0]+0.002,lat_lims[0]+0.002],xycoords='data',backgroundcolor='white',fontsize=16)
    a.set_axis_off()
ax10.annotate('B', xy=[0.01,0.01],xytext=[0.01,0.01],xycoords='axes fraction',backgroundcolor='white',fontsize=16)
ax10.set_axis_off()
ax20.annotate('C', xy=[0.01,0.01],xytext=[0.01,0.01],xycoords='axes fraction',backgroundcolor='white',fontsize=16)
ax20.set_axis_off()

ax05_cmap = fig.add_axes([0.9,0.67,0.01,0.33])
ax05_cmap.set_axis_off()

fake_data = ax05.imshow(np.random.random((10,2)),cmap=colormap, vmin=-1,vmax=1,alpha = 1)



ax05_cax = add_right_cax(ax05,pad=0.01,width=0.01)
cbar0 = fig.colorbar(fake_data, cax=ax05_cax, orientation='vertical',)
ticklabs = cbar0.ax.get_yticklabels()
cbar0.ax.set_yticklabels(ticklabs, fontsize=16)
cbar0.set_label( label='LOS change', fontsize=16)
cbar0.set_ticks(ticks=[-1, 0, 1], labels=[r'-λ', '0', r'+λ'])
ax15_cax = add_right_cax(ax15,pad=0.01,width=0.01)
cbar1 = fig.colorbar(CSK_r_off_map, cax=ax15_cax, orientation='vertical', )
ticklabs = cbar1.ax.get_yticklabels()
cbar1.ax.set_yticklabels(ticklabs,fontsize=16)
cbar1.set_label(label='Range offset [m]', size=16)
cbar1.set_ticks(ticks=[-3, 0, 3],labels=['-3','0','3'])
ax25_cax = add_right_cax(ax25,pad=0.01,width=0.01)
cbar2 = fig.colorbar(CSK_a_off_map, cax=ax25_cax, orientation='vertical')
ticklabs = cbar2.ax.get_yticklabels()
print(ticklabs)
cbar2.ax.set_yticklabels(ticklabs,fontsize=16)
cbar2.set_label(label= 'Azimuth offset [m]',size=16)
cbar2.set_ticks(ticks=[-3, 0, 3],labels=['-3','0','3'])



# set titles
ax01.set_title('Sentinel-1\n20200908-20201119\nBperp: 11 m ')
ax02.set_title('CSK Pair 1\n20200927-20201113\nBperp: 4 m')
ax03.set_title('CSK Pair 2\n20200926-20201113\nBperp: 623 m')
ax04.set_title('CSK Pair 3\n20200927-20210812\nBperp: 1 m')
ax05.set_title('CSK Pair 4\n20210217-20210218\nBperp: 89 m')


TypeError: plot() got an unexpected keyword argument 'x'

In [ ]:
plt.close('all')

In [27]:
import matplotlib.patches as mpatches
import matplotlib.transforms as mtransforms

def add_right_cax(ax, pad, width):
    axpos = ax.get_position()
    caxpos = mtransforms.Bbox.from_extents(
        axpos.x1 + pad,
        axpos.y0,
        axpos.x1 + pad + width,
        axpos.y1
    )
    cax = ax.figure.add_axes(caxpos)

    return cax

from matplotlib.colors import LinearSegmentedColormap
clist = ['#f579cd', '#f67fc6', '#f686bf', '#f68cb9', '#f692b3', '#f698ad',
                     '#f69ea7', '#f6a5a1', '#f6ab9a', '#f6b194', '#f6b78e', '#f6bd88',
                     '#f6c482', '#f6ca7b', '#f6d075', '#f6d66f', '#f6dc69', '#f6e363',
                     '#efe765', '#e5eb6b', '#dbf071', '#d0f477', '#c8f67d', '#c2f684',
                     '#bbf68a', '#b5f690', '#aff696', '#a9f69c', '#a3f6a3', '#9cf6a9',
                     '#96f6af', '#90f6b5', '#8af6bb', '#84f6c2', '#7df6c8', '#77f6ce',
                     '#71f6d4', '#6bf6da', '#65f6e0', '#5ef6e7', '#58f0ed', '#52e8f3',
                     '#4cdbf9', '#7bccf6', '#82c4f6', '#88bdf6', '#8eb7f6', '#94b1f6',
                     '#9aabf6', '#a1a5f6', '#a79ef6', '#ad98f6', '#b392f6', '#b98cf6',
                     '#bf86f6', '#c67ff6', '#cc79f6', '#d273f6', '#d86df6', '#de67f6',
                     '#e561f6', '#e967ec', '#ed6de2', '#f173d7']
colormap = LinearSegmentedColormap.from_list('dismph', clist, N=256)
#colormap = self.cmap_map(lambda x: x/2 + 0.5, colormap)  # brighten colormap
#colormap = self.cmap_map(lambda x: x*0.75, colormap)     # darken colormap
colormap.set_bad('w', 0.0)

dismph = colormap(range(255))
dismph2 = np.row_stack((np.array(dismph),np.array(dismph)))
print(dismph2)

In [9]:
## overview figure

# 1   2   3  
# 4   5   6   
# 7   8   9   

# 1: True colour S2
# 2: CSK Amplitude
# 3: S1 Amplitude
# 4: CSK ifg
# 5: CSK range off
# 6: CSK azi off
# 7: S1 ifg
# 8: S1 range off
# 9: S1 azi off

%matplotlib osx

from matplotlib.patches import Rectangle
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
# make figure
grid_size = 1000
cmap=cm.vik
clims = [-3, 3]
lon_lims = [110.425, 110.46]
lat_lims = [-7.555, -7.515]
orientation = 'vertical'

CSK_obj = example_pairs[0].Stack[1]

fig = plt.figure()

gs = GridSpec(3, 3, figure=fig)
ax00 = fig.add_subplot(gs[0, 0]) # Merapi true colour with inset
ax01 = fig.add_subplot(gs[0, 1]) # CSK amplitude (with summit inset?)
ax02 = fig.add_subplot(gs[0, 2]) # S1 amplitude (with summit inset?)
ax10 = fig.add_subplot(gs[1, 0]) # CSK ifg
ax11 = fig.add_subplot(gs[1, 1]) # CSK range off
ax12 = fig.add_subplot(gs[1, 2]) # CSK azi off
ax20 = fig.add_subplot(gs[2, 0]) # S1 ifg
ax21 = fig.add_subplot(gs[2, 1]) # S1 range off
ax22 = fig.add_subplot(gs[2, 2]) # S1 azi off



ax00.imshow(TRUE_COL,extent=TRUE_COL_extent)
ax00_1 = ax00.inset_axes([0.5,0.75,0.5,0.25])
ax00_1.add_patch(Rectangle((90,-15),55,25,color='white'))
# world.plot(ax=ax00_1,color='lightgrey',edgecolor='black')
ind_coastlines.plot(ax=ax00_1,color='lightgrey',edgecolor='black')
ax00_1.scatter(110.4,-7.54,s=5,c='red')
ax00_1.set_aspect('equal','box')
ax00_1.set_xlim((105,117))
ax00_1.set_ylim((-9,-5))
ax00_1.set_axis_off()


# S1_PWR_map = ax01.imshow(S1_PWR[250:1000,750:2000],cmap='Greys')
# ax01_1 = inset_axes(ax01,width='50%',height='50%',loc='upper right')
# ax01_1.imshow(S1_PWR[520:625,1100:1310],cmap='Greys')
ax01.imshow(S1_PWR[470:675,1075:1335],cmap='Greys')


# CSK_PWR_map = ax02.imshow(CSK_PWR,cmap='Greys')
# ax02_1 = inset_axes(ax02,width='50%',height='50%',loc='upper right')
# ax02_1.imshow(CSK_PWR[3900:5210,1940:3270],cmap='Greys')
ax02.imshow(CSK_PWR[3400:5710,1440:3770],cmap='Greys')
# ax02_1 = inset_axes(ax02,width='50%',height='50%',loc='lower left')
# ax02_1.imshow(CSK_PWR[4100:4300,2190:2390],cmap='Greys')

ax10.imshow(S1_IFG,extent=S1_IFG_extent)
S1_r_off_map = ax11.hexbin(S1_data[10],S1_data[9],C=S1_data[7],gridsize=grid_size,cmap=cmap,vmin=clims[0],vmax=clims[1])
S1_a_off_map = ax12.hexbin(S1_data[10],S1_data[9],C=S1_data[8],gridsize=grid_size,cmap=cmap,vmin=clims[0],vmax=clims[1])

ax20.imshow(CSK_IFG,extent=CSK_IFG_extent)

CSK_r_off_map = ax21.hexbin(CSK_obj.Lon_off.flatten(),CSK_obj.Lat_off.flatten(),C=CSK_obj.R_off.flatten(),gridsize=grid_size,cmap=cmap,vmin=clims[0],vmax=clims[1])
# plt.colorbar(r_off_map,label='range offset [m]',orientation=orientation)

CSK_a_off_map = ax22.hexbin(CSK_obj.Lon_off.flatten(),CSK_obj.Lat_off.flatten(),C=CSK_obj.A_off.flatten(),gridsize=grid_size,cmap=cmap,vmin=clims[0],vmax=clims[1])
# plt.colorbar(a_off_map,label='azimuth offset [m]',orientation=orientation)


for a in [ax00,ax10,ax11,ax12,ax20,ax21,ax22]:
    a.set_aspect('equal','box')
    a.set_xlim(lon_lims)
    a.set_ylim(lat_lims)
    a.add_artist(ScaleBar(sm.plot.get_1deg_dist(),location='upper left'))

for a,s in zip([ax00,ax01,ax02,ax10,ax11,ax12,ax20,ax21,ax22],['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R']):
    a.set_axis_off()
    a.annotate(s,xy=[0,0],xytext=[0,0],xycoords='figure fraction',)



add_right_cax(ax05)

NameError: name 'CSK_IFG' is not defined

Traceback (most recent call last):
  File "/Applications/anaconda3/envs/PhD/lib/python3.8/site-packages/matplotlib/backend_bases.py", line 1226, in _on_timer
    ret = func(*args, **kwargs)
  File "/Applications/anaconda3/envs/PhD/lib/python3.8/site-packages/matplotlib/backends/backend_macosx.py", line 68, in callback_func
    callback()
  File "/Applications/anaconda3/envs/PhD/lib/python3.8/site-packages/matplotlib/backends/backend_macosx.py", line 88, in _draw_idle
    self.draw()
  File "/Applications/anaconda3/envs/PhD/lib/python3.8/site-packages/matplotlib/backends/backend_macosx.py", line 50, in draw
    super().draw()
  File "/Applications/anaconda3/envs/PhD/lib/python3.8/site-packages/matplotlib/backends/backend_agg.py", line 400, in draw
    self.figure.draw(self.renderer)
  File "/Applications/anaconda3/envs/PhD/lib/python3.8/site-packages/matplotlib/artist.py", line 95, in draw_wrapper
    result = draw(artist, renderer, *args, **kwargs)
  File "/Applications/anaconda3/envs/

In [2]:
import mintpy


from mintpy.colors import ColormapExt
mph = ColormapExt('dismph').colormap
print(mph)

ModuleNotFoundError: No module named 'mintpy.colors'

In [ ]:
plt.close('all')

In [10]:
%matplotlib osx

from matplotlib.patches import Rectangle
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
# make figure
grid_size = 1000
cmap=cm.vik
clims = [-3, 3]
lon_lims = [110.425, 110.46]
lat_lims = [-7.555, -7.515]
orientation = 'vertical'

obj = example_pairs[0].Stack[1]

fig = plt.figure()

gs = GridSpec(2, 4, figure=fig)
ax00 = fig.add_subplot(gs[0, 0]) # Merapi true colour with inset
ax10 = fig.add_subplot(gs[1, 0]) # Merapi NDVI

ax01 = fig.add_subplot(gs[0, 1]) # Merapi elevation
ax11 = fig.add_subplot(gs[1, 1]) # Merapi slope
ax02 = fig.add_subplot(gs[0, 2]) # Merapi shadow + overlay
ax12 = fig.add_subplot(gs[1, 2]) # Merapi ifg
ax03 = fig.add_subplot(gs[0, 3]) # range offset
ax13 = fig.add_subplot(gs[1, 3]) # azimuth offset


ax00.imshow(TRUE_COL,extent=TRUE_COL_extent)
ax00_1 = inset_axes(ax00,width='50%',height='25%',loc=2)
ax00_1.add_patch(Rectangle((90,-15),55,25,color='white'))
world.plot(ax=ax00_1,color='lightgrey',edgecolor='black')
ax00_1.scatter(110.4,-7.54,s=5,c='red')
ax00_1.set_aspect('equal','box')
ax00_1.set_xlim((90,145))
ax00_1.set_ylim((-15,10))
ax00_1.set_axis_off()

ndvi_map = ax10.imshow(NDVI,cmap=ndvi_cmap,extent=NDVI_extent,vmin=-1,vmax=1)
plt.colorbar(ndvi_map,label='NDVI [-]',orientation=orientation)

dem_map = ax01.imshow(DEM_heights,cmap='gist_earth',extent=DEM_extent,vmin=0,vmax=3000)
ax01.imshow(SHADING, cmap=cm.grayC, alpha=0.3 , extent=DEM_extent)
plt.colorbar(dem_map,label='elevation [m]',orientation=orientation)

slope_map = ax11.imshow(DEM_slope,cmap='Greys_r',extent=DEM_extent,vmin=0,vmax=45)
plt.colorbar(slope_map,label='slope [deg.]', extend='max',orientation=orientation)

ax02.imshow(ls_map,cmap='Greys',extent=DEM_extent)
ax02.scatter([],[],s=30,c='black',edgecolor='black',label='Shadow')
ax02.scatter([],[],s=30,c='white',edgecolor='black',label='Overlay')
ax02.scatter([],[],s=30,c='grey',edgecolor='black',label='Normal or\nforeshortening')
ax02.legend(loc='best')

ax12.imshow(IFG,extent=IFG_extent)


# ax03.imshow(SHADING,cmap=cm.grayC,alpha=0.5, extent=DEM_extent)
r_off_map = ax03.hexbin(obj.Lon_off.flatten(),obj.Lat_off.flatten(),C=obj.R_off.flatten(),gridsize=grid_size,cmap=cmap,vmin=clims[0],vmax=clims[1])
plt.colorbar(r_off_map,label='range offset [m]',orientation=orientation)

# ax13.imshow(SHADING,cmap=cm.grayC,alpha=0.5, extent=DEM_extent)
a_off_map = ax13.hexbin(obj.Lon_off.flatten(),obj.Lat_off.flatten(),C=obj.A_off.flatten(),gridsize=grid_size,cmap=cmap,vmin=clims[0],vmax=clims[1])
plt.colorbar(a_off_map,label='azimuth offset [m]',orientation=orientation)

print(lon_lims,lat_lims)

for (ax,text) in zip([ax00,ax10,ax01,ax11,ax02,ax12,ax03,ax13],['A','B','C','D','E','F','G','H']):
    ax.add_artist(ScaleBar(sm.plot.get_1deg_dist(),location='upper right'))
    if ax ==ax00:
        ax.text(110.44570,-7.52014,text,fontsize=20,backgroundcolor=(1,1,1,0.3))
    else:
        ax.text(110.42633,-7.52014,text,fontsize=20,backgroundcolor=(1,1,1,0.3))
    ax.set_xlim(lon_lims)
    ax.set_ylim(lat_lims)
    ax.set_axis_off()

# fig.tight_layout()

[110.425, 110.46] [-7.555, -7.515]


2023-09-21 16:06:00.545 python[86118:1940784] +[CATransaction synchronize] called within transaction
2023-09-21 16:06:07.086 python[86118:1940784] +[CATransaction synchronize] called within transaction
2023-09-21 16:06:08.220 python[86118:1940784] +[CATransaction synchronize] called within transaction


In [7]:
# plt.close('all')